In [ ]:
pip install ccxt

     |████████████████████████████████| 2.0MB 7.8MB/s 
     |████████████████████████████████| 296kB 38.7MB/s 
     |████████████████████████████████| 3.2MB 39.5MB/s 
     |████████████████████████████████| 1.3MB 48.7MB/s 
     |████████████████████████████████| 143kB 48.9MB/s 
     |████████████████████████████████| 296kB 44.2MB/s 


In [ ]:
pip install schedule

In [ ]:
pip install config

In [ ]:
pip install python-binance

     |████████████████████████████████| 61kB 4.6MB/s 
     |████████████████████████████████| 112kB 10.6MB/s 
     |████████████████████████████████| 184kB 11.9MB/s 
     |████████████████████████████████| 286kB 12.8MB/s 


In [ ]:
import ccxt
import config
import schedule
import pandas as pd
import smtplib

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
from datetime import datetime
import time

In [ ]:
from binance import Client

In [ ]:
api_key = '*******************************'
secret_key = '****************************'

In [ ]:
exchange = ccxt.binanceus()
bars = exchange.fetch_ohlcv('ETH/USDT', timeframe='1h', limit=30)

In [ ]:
for bar in bars:
  print(bar)

In [ ]:
def tr(data):
    data['previous_close'] = data['close'].shift(1)
    data['high-low'] = abs(data['high'] - data['low'])
    data['high-pc'] = abs(data['high'] - data['previous_close'])
    data['low-pc'] = abs(data['low'] - data['previous_close'])

    tr = data[['high-low', 'high-pc', 'low-pc']].max(axis=1)

    return tr

In [ ]:
def atr(data, period):
    data['tr'] = tr(data)
    atr = data['tr'].rolling(period).mean()

    return atr

In [ ]:
def supertrend(df, period=10, atr_multiplier=3):
    hl2 = (df['high'] + df['low']) / 2
    df['atr'] = atr(df, period)
    df['upperband'] = hl2 + (atr_multiplier * df['atr'])
    df['lowerband'] = hl2 - (atr_multiplier * df['atr'])
    df['in_uptrend'] = True

    for current in range(1, len(df.index)):
        previous = current - 1

        

        if df['close'][current] > df['upperband'][previous]:
            df['in_uptrend'][current] = True
        elif df['close'][current] < df['lowerband'][previous]:
            df['in_uptrend'][current] = False
        else:
            df['in_uptrend'][current] = df['in_uptrend'][previous]

            if df['in_uptrend'][current] and df['lowerband'][current] < df['lowerband'][previous]:
                df['lowerband'][current] = df['lowerband'][previous]

            if not df['in_uptrend'][current] and df['upperband'][current] > df['upperband'][previous]:
                df['upperband'][current] = df['upperband'][previous]
        
    return df


In [ ]:
in_position = False

In [ ]:
def sendMail(text):
  server = smtplib.SMTP('smtp.gmail.com', 587)
  server.starttls()
  server.login('****************@mail.com', 'pass******************')
  SUBJECT = 'NOTIFICATION for new SIGNAL'
  TEXT = text
  message = 'Subject: {}\n\n{}'.format(SUBJECT, TEXT)
  server.sendmail('abcd@gmail.com',
                'xyz@gmail.com',
                message
                )
  server.quit()

In [ ]:
def check_buy_sell_signals(df):
    global in_position

    print("checking for buy and sell signals")
    print(df.tail(3))
    last_row_index = len(df.index) - 1
    previous_row_index = last_row_index - 1

    if not df['in_uptrend'][previous_row_index] and df['in_uptrend'][last_row_index] and df['close']['last_row_index'] > df['ema']['last_row_index']:
        print("changed to uptrend, buy")
        if not in_position:
            print("Time to buy! Email sent!!")
            message = str(df[last_row_index])
            sendMail(message)
            in_position = True
        else:
            print("already in position, nothing to do")
    
    if df['in_uptrend'][previous_row_index] and not df['in_uptrend'][last_row_index]:
        if in_position:
            print("changed to downtrend, sell")
            message = 'Sell, previous Signal now in Downtrend'
            sendMail(message)
            in_position = False
        else:
            print("You aren't in position, nothing to sell")

In [ ]:
def run_bot():
    print(f"Fetching new bars for {datetime.now().isoformat()}")
    bars = exchange.fetch_ohlcv('DOGE/USDT', timeframe='1m', limit=202)
    df = pd.DataFrame(bars[:-1], columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    # creating EMA200
    df['ema'] = pd.Series.ewm(df['close'], span= 200).mean()
    #for col in df.columns:
    # print(col)

    supertrend_data = supertrend(df)
    
    check_buy_sell_signals(supertrend_data)
    #return df

In [ ]:
#run_bot()

In [ ]:
schedule.every(40).seconds.do(run_bot)


while True:
    schedule.run_pending()
    time.sleep(5)

Fetching new bars for 2021-05-21T00:58:01.083690
checking for buy and sell signals
              timestamp      open      high  ...  upperband  lowerband  in_uptrend
198 2021-05-21 00:54:00  0.408072  0.409119  ...   0.413095   0.405218        True
199 2021-05-21 00:55:00  0.408800  0.408820  ...   0.412817   0.405218        True
200 2021-05-21 00:56:00  0.408166  0.410258  ...   0.413520   0.405218        True

[3 rows x 16 columns]
Fetching new bars for 2021-05-21T00:58:41.359036
checking for buy and sell signals
              timestamp      open      high  ...  upperband  lowerband  in_uptrend
198 2021-05-21 00:55:00  0.408800  0.408820  ...   0.412817   0.405218        True
199 2021-05-21 00:56:00  0.408166  0.410258  ...   0.413520   0.405218        True
200 2021-05-21 00:57:00  0.410200  0.410200  ...   0.413757   0.405617        True

[3 rows x 16 columns]
Fetching new bars for 2021-05-21T00:59:21.552560
checking for buy and sell signals
              timestamp      open      hi

KeyError: ignored